<p align="center">
  <img width="100" height="100" src="../../images/logo.png">
</p>

<div>
<h1>022. Preproceso: Modelo C_SEV</h1> 

Canadian Car Accidents Practice <br>
<strong>Aprendizaje Automático</strong> <br>
<strong>Master Universitario en Ciencia de Datos<strong>
</div>

<div style='text-align:right'>Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div>
<div style='text-align:right'>Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>

---

## 22.0 Introducción

En este notebook realizaremos el preprocesado de los datos del dataset conforme al análisis realizado en el notebook EDA 
```01_EDA```. Para ello procederemos a explicar paso a paso las decisiones tomadas en cuanto a su preprocesado.

Para este trabajo estamos utilizando un entorno de propósito espécifico.

In [2]:
# Verificamos el entorno: ML_P1
!conda info


     active environment : ML_P1
    active env location : C:\Users\carviagu\anaconda3\envs\ML_P1
            shell level : 2
       user config file : C:\Users\carviagu\.condarc
 populated config files : 
          conda version : 4.10.3
    conda-build version : 3.21.4
         python version : 3.8.8.final.0
       virtual packages : __cuda=11.3=0
                          __win=0=0
                          __archspec=1=x86_64
       base environment : C:\Users\carviagu\anaconda3  (writable)
      conda av data dir : C:\Users\carviagu\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/no

### 22.0.1 Estructura

<< PONER ESTRUCTURA >>

## Librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

%matplotlib inline

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

## Scripts
En este notebook procedemos a importar dos scripts con funciones útiles, previamente utilizadas en el EDA y que permitirán verificar el proceso de realización de las muestras de train y test.

In [2]:
import sys

import sys  
sys.path.insert(0, '../../scripts/')

import csv_tools
import eda_tools

## 22.1 Lectura y preparación

A continuación leeremos el dataset, de la misma forma que lo hicimos en el EDA.

In [3]:
# Función propia que verifica que existe el fichero de datos previamente.
accidents_df = csv_tools.csv_import(origin="../../data/NCDB_1999_to_2014.csv")

Reading file...
Reading ended.


In [4]:
accidents_df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3


## 22.2 Balanceo de ```C_SEV```
En el EDA pudimos apreciar como había un importante desequilibrio en la variable objetivo, existiendo un 98% y 2% de accidentes no mortales y mortales respectivamente. 

In [5]:
accidents_df['C_SEV'].value_counts()

2    5761772
1      98633
Name: C_SEV, dtype: int64

Para ello realizaremos un remuestreo de los datos mediante la técnica de **subsampling** con el objetivo de reducir el número de muestras de la clase 2 (ningún fallecido) frente a las de 1 (al menos un fallecido). Para ello:
* Primero, dividimos el dataset dos dataset, uno por clase.
* Segundo, realizamos un resample de la clase mayoritaria, para extraer una muestra del mismo. 
* Finalmente, juntaremos ambos datasets para formar la muestra a utilizar. 

In [6]:
# Dividimos la clases
sev01_df = accidents_df[accidents_df['C_SEV'] == 1]
sev02_df = accidents_df[accidents_df['C_SEV'] == 2]

Realizamos una muestra de ```sev02_df``` con un tamaño superior al 5% de ```sev01_df```. Esto se realiza para que pese a reducir el desequilibrio existente, buscamos que se asemeje a la realidad y por lo tanto mantenga cierto grado de desequilibrio. 

Destacar que usamos como semilla el valor 0. Este se ha determinado de forma arbitraria, permitirá replicar los resultados en caso de querer realizar más pruebas.

In [7]:
# Tamaño a muestrear
sample_size = int(sev01_df.shape[0] * (1.05))

# Realizamos una muestra del sev02_df
sev02_sam_df = resample(sev02_df, replace=True, n_samples = sample_size, random_state=0)

# Juntamos y creamos la versión a utilizar
accidents_eq_df = pd.concat([sev01_df, sev02_sam_df])
accidents_eq_df['C_SEV'].value_counts()

2    103564
1     98633
Name: C_SEV, dtype: int64

Observamos como se ha reducido considerablemente la muestra de accidentes no mortales, y hemos podido equilibrar en cierto grado el dataset. 

Ahora, guardaremos esta versión reducida del dataset para las siguientes tareas.

In [9]:
# Guardar resultados (CUIDADO: sobreescritura del fichero)
# accidents_eq_df.to_csv("../../data/NCBD_reduced.csv")

## 22.3 Train y Test
A continuación vamos a proceder a crear y dividir los datos en Train (muestra de entrenamiento) y test (muestra de test) que usaremos para nuestros modelos. 

In [9]:
# accidents_eq_df = csv_tools.csv_import(origin = "../../data/NCBD_reduced.csv")

# Separamos los valores (X) de la variable objetivo (Y)
X = accidents_eq_df.drop('C_SEV', axis=1)
Y = accidents_eq_df['C_SEV']

# Realizamos la división de train y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## 22.4 Tratamiento
A continuación procederemos a realizar la limpieza de los datos y su procesamiento:

### 22.4.1 Filtrado inicial de variables y valores anómalos
Si bien nos detendremos en datalle a analizar la importancia de las variables para el modelo. Existen a prior unas variables que no son útiles y que por lo tanto eliminaremos del mismo. Estas son: 
* V_ID: Identificador de vehículo
* P_ID: Identificador de persona
* P_ISEV: Datos sobre si la persona ha fallecido o no

In [11]:
X_train = X_train.drop(['V_ID', 'P_ID', 'P_ISEV'], axis=1)

Por otro lado, en el EDA pudimos observar que existian vehículos con fecha de fabricación 2015, cuando los accidentes son hasta 2014. Procedemos a eliminarlos en el caso de que continuen en la muestra de train.

In [15]:
X_train = X_train.drop(X_train[X_train['V_YEAR'] == '2015'].index, axis=0)

### 22.4.2 Valores faltantes
Para llevar a cabo este proceso realizaremos la conversión correspondiente por tipo de variable, esto es, pues el signficado que estas tienen es distinto. 

Primero veamos como está la muestra de train:

In [12]:
especial_values = [['U', 'UU', 'UUUU'], ['N', 'NN', 'NNNN'], ['Q', 'QQ', 'QQQQ']]
eda_tools.special_values_summary(df = X_train, vals = especial_values)

,column_name,U,U%,N,N%,Q,Q%,total,total%
0,C_YEAR,0,[0.0],0,[0.0],0,[0.0],0,0.0
1,C_MNTH,8,[0.005],0,[0.0],0,[0.0],8,0.0
2,C_WDAY,30,[0.019],0,[0.0],0,[0.0],30,0.0
3,C_HOUR,1657,[1.024],0,[0.0],0,[0.0],1657,1.0
4,C_VEHS,16,[0.01],0,[0.0],0,[0.0],16,0.0
5,C_CONF,4555,[2.816],0,[0.0],8276,[5.116],12831,7.9
6,C_RCFG,11779,[7.282],0,[0.0],4503,[2.784],16282,10.1
7,C_WTHR,1867,[1.154],0,[0.0],423,[0.262],2290,1.4
8,C_RSUR,1765,[1.091],0,[0.0],5017,[3.102],6782,4.2
9,C_RALN,8780,[5.428],0,[0.0],616,[0.381],9396,5.8


**Para los valores U (desconocido)**

- **Primero.** Calcularemos para cada grupo que categoría concentra más del 80% de valores, es decir, asignaremos un valor mayoritario. Asumimos que si la mayoría de los datos son de esa categoría los faltantes también lo serán.

- **Segundo.** En el caso de no existir una categoría que concentre más del 80% se establecerá el valor medio (redondeado a la unidad superior) de las categorías asignadas. Al haber tanta variabilidad asignaremos la categoría media.

In [13]:
# VALORES FALTANTES
# TODO 
#
# U -> a NA
# Asignar valor medio o mayoritario (en función de la variable)
#
# Q -> crear una clase nueva de la categoría
# 
# N -> crear una clase nueva de la categoría (para C_SEV)
#

In [14]:
# VALORES EXTRAÑOS
# Eliminar valores extraños (año 2015?)

In [15]:
# ENCODING ¿?
# Sexo -> M/F LabelEncoding o OneHotEncoding
# 
# Encoding de variables con una codificación extraña  
#

In [16]:
# SELECCIÓN DE VARIABLES
# Correlación ?
# Lasso o modelo de regresión?
# Justificar elección final ? 

---

<div style='text-align:center'>Elaborado por Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div> 
<div style='text-align:center'>y Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div> 